# Initialization

In [55]:
from src.SIV_library.lib import SIV, OpticalFlow
from src.SIV_library.advanced import Warp, CTF, Refine

import os
import sys
import torch
import matplotlib.pyplot as plt

import pandas as pd
from datetime import datetime, timedelta

plt.rcParams["animation.html"] = "jshtml"
%matplotlib notebook
# %matplotlib inline

os.environ["OPENCV_IO_ENABLE_OPENEXR"]="1"
sys.path.append("../../")

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print('device:', device)

if device == "cuda":
    torch.cuda.empty_cache()
    
folder = r"../Test Data/Full big vid PROCESSED"

device: cuda


# Matching

In [8]:
mode = 1

siv_gen = SIV(
    folder=folder, 
    device=device, 
    window_size=128, 
    overlap=64, 
    search_area=(50, 0, 0, 0), 
    mode=mode, 
    num_passes=2, 
    scale_factor=0.5
)

In [17]:
idx = 0
u1_avg = torch.zeros((len(siv_gen.dataset),))

for res in siv_gen():
    x, y, u, v = res
    
    warp = Warp(x, y, u, v)
    warp.interpolate_field(siv_gen.dataset.img_shape)
    u1_avg[idx] = torch.mean(warp.u)

    idx += 1

SAD: 100%|██████████| 2675/2675 [07:43<00:00,  5.77it/s]


Interpolate velocity field:

# Optical flow

In [24]:
opti_gen = OpticalFlow(
    folder=folder, 
    device=device, 
    alpha=100., 
    num_iter=1000, 
    eps=0.
)

In [25]:
idx = 0
u2_avg = torch.zeros((len(opti_gen.dataset),))

for res in opti_gen():
    x, y, u, v = res
    u2_avg[idx] = torch.mean(u)
    idx += 1

Optical flow: 100%|██████████| 2675/2675 [49:54<00:00,  1.12s/it]


Course-to-fine optical flow:

In [52]:
ctf_gen = CTF(opti_gen, num_passes=4, scale_factor=1/2)

In [53]:
idx = 0
u3_avg = torch.zeros((len(ctf_gen.dataset),))

for res in ctf_gen():
    x, y, u, v = res
    u3_avg[idx] = torch.mean(u)
    idx += 1

  0%|          | 4/2675 [00:16<2:59:32,  4.03s/it]


KeyboardInterrupt: 

# Hybrid method (matching + optical flow)

In [59]:
refine_gen = Refine(siv_gen, opti_gen)

In [79]:
idx = 0
u4_avg = torch.zeros((len(refine_gen.dataset),))

for res in refine_gen():
    x, y, u, v = res
    u4_avg[idx] = torch.mean(u)
    idx += 1

100%|██████████| 2675/2675 [57:10<00:00,  1.28s/it] 


# Loading data

In [80]:
def get_datetimes(folder):
    time_objects = []
    
    for file in os.listdir(folder):
        name_formatted = (file[:-4].split("_"))
        name_formatted[-1] = f"{name_formatted[-1]}000"
        name_formatted = [name_formatted[2], name_formatted[1], name_formatted[0], *name_formatted[3:]]
        time = ':'.join(name_formatted[3:])
        time_objects.append(datetime.strptime(time, '%H:%M:%S:%f') - timedelta(seconds=3.736))
    return time_objects

ds = get_datetimes(folder)
start = ds[0]
ds = [(t - start).total_seconds() for t in ds]

In [81]:
testo = pd.read_csv("../Test Data/testo.csv", sep=';')
testo = testo.dropna(how='all').to_numpy()[:, :2]

times = [datetime.strptime(time, '%H:%M:%S') for time in testo[:, 0].flatten()]
velocities = [float(u.replace(',', '.')) for u in testo[:, 1].flatten()]

times = [(t - start).total_seconds() for t in times]

# Plotting results

In [82]:
def process(avg):
    pair_averages = avg[::2].cpu()
    
    k = 10  # window size of moving median filter
    min_idx, max_idx = 0, len(pair_averages) - 1
    meds = torch.zeros((len(pair_averages),))
    
    for idx, avg in enumerate(pair_averages):
        lower, upper = idx - k, idx + k
        lower = min_idx if lower < min_idx else lower
        upper = max_idx if upper > max_idx else upper
    
        sliced = pair_averages[lower:upper+1]
        meds[idx] = torch.median(sliced)
    return meds

In [83]:
scale = 114.75/30  # px / mm
fps = 198.53

factor = fps/(scale * 1000)

In [84]:
u1, u2, u4 = process(-u1_avg), process(-u2_avg), process(-u4_avg)  # u3 = process(-u3_avg)
u1, u2, u4 = u1 * factor, u2 * factor, u4 * factor  # u3 = u3 * factor

In [86]:
plt.close('all')
    
# vs = [v for i, v in enumerate(velocities) if ds[0] <= times[i] <= ds[-1]]
# avg_vs = sum(vs)/len(vs)
# correction = avg_vs / torch.mean(meds).item()

# plt.plot(ds[::2], pair_averages, label='measurements')
plt.plot(ds[::2], u1, 'b-', label='SAD')
plt.plot(ds[::2], u2, 'g-', label='Optical')
plt.plot(ds[::2], u4, 'b--', label='Refine')
plt.plot(times, velocities, 'r--', label='Testo', linewidth=1)

plt.xlabel('Time (s)')
plt.ylabel('Velocity (m/s)')

plt.xlim(left=ds[0], right=ds[-1])
plt.ylim(bottom=0.)
plt.title('Velocity time series of anemometer compared to multipass SAD')

plt.legend()
plt.show()

<IPython.core.display.Javascript object>

In [240]:
vx = -u1[::2] * factor
std = torch.std(vx)

print(f"std \t{round(std.item(), 2)}")

std 	1.13


In [87]:
import numpy as np
np.save("../experiments/SAD.npy", u1_avg.numpy())
np.save("../experiments/optical.npy", u2_avg.numpy())
np.save("../experiments/refine.npy", u4_avg.numpy())

In [89]:
u1_avg = torch.tensor(np.load("../experiments/SAD.npy"))
u1_avg

tensor([-17.8040, -38.4342, -17.1759,  ..., -11.6538, -35.1645,  -8.6250])